<a href="https://colab.research.google.com/github/1C3K1N6/1C3K1N6/blob/main/10%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0%D0%9F%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D0%B5%D0%A2%D0%BE%D1%80%D0%B3%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Загрузка данных (пример для данных из строки)
torg = '/content/drive/MyDrive/adre.us.csv'
df = pd.read_csv(torg) # wget скачал в рабочую директорию файл с датасетом и мы его уже можем использовать

torg2 = '/content/drive/MyDrive/agz.us.csv'
df2 = pd.read_csv(torg2)
#df = pd.read_csv(pd.compat.StringIO(data), parse_dates=['Date'], index_col='Date')
#df = pd.read_csv(data, parse_dates=['Date'], index_col='Date')

# Выбор признака для прогноза (цена закрытия) agz.us.csv


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])
df.head().info()
df2.head().info()
df.head()
data = df[['Close']].values
data2 = df2[['Close']].values

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Close'], label='Цена закрытия')
plt.title('Исторические данные цены закрытия (1ый)')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df2['Date'], df2['Close'], label='Цена закрытия')
plt.title('Исторические данные цены закрытия (2ой)')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.legend()
plt.show()

In [ ]:
# Нормализация данных
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Создание временных окон
def create_dataset(df, window=60):
    X, y = [], []
    for i in range(window, len(df)):
        X.append(df[i-window:i, 0])
        y.append(df[i, 0])
    return np.array(X), np.array(y)

window = 60
X, y = create_dataset(scaled_data, window)

# Разделение на обучающую и тестовую выборки
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Преобразование в 3D-тензор для LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

In [ ]:
# Предсказание на тестовых данных
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index[-len(y_test):], y_test_actual, label='Реальные значения')
plt.plot(df.index[-len(y_test):], predictions, label='Предсказанные значения')
plt.title('Сравнение реальных и предсказанных значений')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия')
plt.legend()
plt.show()

In [ ]:
#scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data2 = scaler.fit_transform(data2)

# Создание временных окон
def create_dataset2(df2, window=60):
    X, y = [], []
    for i in range(window, len(df2)):
        X.append(df2[i-window:i, 0])
        y.append(df2[i, 0])
    return np.array(X), np.array(y)

window = 60
X, y = create_dataset(scaled_data2, window)

# Разделение на обучающую и тестовую выборки
split = int(0.8 * len(X))
X_test2 =  X[split:]
y_test2 = y[split:]

# Преобразование в 3D-тензор для LSTM

X_test2 = np.reshape(X_test2, (X_test2.shape[0], X_test2.shape[1], 1))

In [ ]:
predictions2 = model.predict(X_test2)
predictions2 = scaler.inverse_transform(predictions2)
y_test_actual2 = scaler.inverse_transform(y_test2.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df2.index[-len(y_test2):], y_test_actual2, label='Реальные значения')
plt.plot(df2.index[-len(y_test2):], predictions2, label='Предсказанные значения')
plt.title('Сравнение реальных и предсказанных значений на другом датасете')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия')
plt.legend()
plt.show()